# Apache Hive

## Instalación de ambiente

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
exec(open('/content/drive/MyDrive/Proyectos/apache-hive-pig/hadoop_colab_installer.py').read())

Active services:
2424 ResourceManager
2568 DataNode
2699 JobHistoryServer
2734 Jps
2494 NameNode
2655 NodeManager



## Actividad 0: Creación de dos directorios en HDFS y tablas

In [3]:
!hdfs dfs -mkdir mp1_df1
!hdfs dfs -put /content/drive/MyDrive/Proyectos/apache-hive-pig/userid-timestamp-artid-artname-traid-traname.tsv mp1_df1

!hdfs dfs -mkdir mp1_df2
!hdfs dfs -put /content/drive/MyDrive/Proyectos/apache-hive-pig/userid-profile.tsv mp1_df2

In [4]:
!hdfs dfs -ls /user/root/mp1_df1

Found 1 items
-rw-r--r--   1 root supergroup 2529193595 2024-11-11 05:50 /user/root/mp1_df1/userid-timestamp-artid-artname-traid-traname.tsv


In [5]:
!hdfs dfs -ls /user/root/mp1_df2

Found 1 items
-rw-r--r--   1 root supergroup      37842 2024-11-11 05:50 /user/root/mp1_df2/userid-profile.tsv


In [6]:
%%writefile create_table1.sql
-- Creación de la tabla para los eventos de escucha
CREATE EXTERNAL TABLE lastfm_listens (
    user_id STRING,
    event_timestamp STRING,
    artist_id STRING,
    artist_name STRING,
    track_id STRING,
    track_name STRING
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t'
STORED AS TEXTFILE
LOCATION '/user/root/mp1_df1';

Writing create_table1.sql


In [7]:
!hive -f create_table1.sql

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.9-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.10.1/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]

Logging initialized using configuration in jar:file:/content/apache-hive-2.3.9-bin/lib/hive-common-2.3.9.jar!/hive-log4j2.properties Async: true
OK
Time taken: 9.232 seconds


In [8]:
%%writefile create_table2.sql
-- Creación de la tabla para los perfiles de usuarios
CREATE EXTERNAL TABLE  lastfm_users (
    user_id STRING,
    gender STRING,
    age INT,
    country STRING,
    registered STRING
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t'
STORED AS TEXTFILE
LOCATION '/user/root/mp1_df2';

Writing create_table2.sql


In [9]:
!hive -f create_table2.sql

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.9-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.10.1/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]

Logging initialized using configuration in jar:file:/content/apache-hive-2.3.9-bin/lib/hive-common-2.3.9.jar!/hive-log4j2.properties Async: true
OK
Time taken: 10.573 seconds


## Actividad 1: Artista más popular

In [10]:
%%writefile actividad1.sql
-- Consulta para el Top-10 de los artistas más escuchados
SELECT artist_name, COUNT(*) AS plays
FROM lastfm_listens
GROUP BY artist_name
ORDER BY plays DESC
LIMIT 10;

Writing actividad1.sql


In [11]:
!hive -f actividad1.sql

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.9-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.10.1/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]

Logging initialized using configuration in jar:file:/content/apache-hive-2.3.9-bin/lib/hive-common-2.3.9.jar!/hive-log4j2.properties Async: true
Query ID = root_20241111055137_bdcc90de-dbe7-4465-b562-4ae724471d4b
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 10
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.

El más popular de este top 10 es Radiohead con 115209 plays y el menos popular del top es Elliott Smith con 50278 plays.

## Actividad 2: Distribución por género

In [12]:
%%writefile actividad2.sql

SELECT gender, COUNT( l.user_id) AS listeners
FROM lastfm_listens l
JOIN lastfm_users u ON l.user_id = u.user_id
WHERE l.artist_name = 'Radiohead' AND gender !=''
GROUP BY gender;

Writing actividad2.sql


In [13]:
!hive -f actividad2.sql

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.9-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.10.1/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]

Logging initialized using configuration in jar:file:/content/apache-hive-2.3.9-bin/lib/hive-common-2.3.9.jar!/hive-log4j2.properties Async: true
Query ID = root_20241111055552_c3412097-7427-488a-bd66-47f2faff4e78
Total jobs = 1
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.9-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.10.1/share/hadoop/common/lib/sl

La cantidad de hombres que han escuchado alguna canción del artista más popular es 63784 y la cantidad de mujeres es 43748.

## Actividad 3: Distribución por edad

In [ ]:
# Inserte su código

In [14]:
%%writefile actividad3.sql

SELECT u.age, COUNT( l.user_id) AS listeners
FROM lastfm_listens l
JOIN lastfm_users u ON l.user_id = u.user_id
WHERE l.artist_name = 'Radiohead' AND u.age is not null
GROUP BY u.age
ORDER BY u.age ASC;


Writing actividad3.sql


In [15]:
!hive -f actividad3.sql

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.9-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.10.1/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]

Logging initialized using configuration in jar:file:/content/apache-hive-2.3.9-bin/lib/hive-common-2.3.9.jar!/hive-log4j2.properties Async: true
Query ID = root_20241111055940_28865feb-198b-4657-8d9c-498b6c8c8a0a
Total jobs = 2
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.9-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.10.1/share/hadoop/common/lib/sl

1543 usuarios de 35 años han escuchado al artista más popular que en este caso es Radiohead.